In [1]:
from tensorflow.keras.models import load_model
import cv2
import numpy as np

In [2]:
model = load_model('facemask_detection.model')

fase_cls = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
source = cv2.VideoCapture(0)

dict_labels = {0:'no mask', 1:'mask'}
dict_color = {0:(0,0,255), 1:(0,255,0)}

In [ ]:
while True:
    
    ret, img = source.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = fase_cls.detectMultiScale(gray, 1.3, 5)
    
    for x,y,w,h in faces:
        
        face_img = gray[y:y+w,x:x+w]
        resized = cv2.resize(face_img, (100, 100))
        normalized = resized/255
        reshaped = np.reshape(normalized, (1, 100, 100, 1))
        result = model.predict(reshaped)
        
        label = 1 if result >= 0.5 else 0
        cv2.rectangle(img, (x, y), (x+w, y+w), dict_color[label], 2)
        cv2.rectangle(img, (x, y-40), (x+w, y), dict_color[label], -1)
        txt = f'{dict_labels[label]} {round(result[0][0], 4)}'
        cv2.putText(img, txt, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)
        
    
    cv2.imshow('LIVE', img)
    key = cv2.waitKey(1)
    
    if key == 27:
        break
    
cv2.destroyAllWindows()
source.release()